In [1]:
import time
import sys
from collections import Counter
import pandas as pd
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import pylab
import gzip
from Bio import SeqIO
import seaborn as sns
import itertools
from Bio.Align import AlignInfo
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [ ]:
#template sequences: DNA sequences of your native template protein(s) that were the basis of an SSM library
XCDP07 = 'GCGGACCCGAAAAAGGTCCTGGACAAAGCGAAAGACCGTGCGGAAAACGTTGTTCGTAAACTCAAGAAGGAACTCGAAGAACTGTACAAAGAGGCGCGTAAGCTGGACCTGACCCAGGAAATGCGTGATCGTATCCGCCTGGCGGCGATTGCAGCGCGTATCGCGGCGTTCGGTGACATCTTCCACGCGATCATGGAGGCGCTGGAGGAAGCGCGCAAGCTGAAAAAAGCGGGTCTGGTTAACTCTCAGCAGCTCGACGAACTCAAACGTCGTCTGGAAGAGCTCGATGAAGAAGCCGCACAACGTGCAGAAAAGCTGGGTAAAGAGTTTGAACTGAAACTGGAATACGGT'
A2CDP06 = 'GCGGACCCGAAAAAAGTTCTGGATAAAGCGAAAGACGAAGCAGAAAACCGTGTTCGTGAACTGAAACAAAAACTCGAAGAACTCTACAAAGAAGCTCGTAAACTGGACCTGACCCAGGAAATGCGTCAGGAACTGGTGGACAAAGCGCGTGCGGCGTCTCTGCAGGCGTCTGGTGACATCTTCTACGCGATCCTGCGTGCGCTCGCGGAAGCGGAGAAACTCAAAAAAGCGGGTCTGGTTAACTCTCAGCAGCTGGACGAGCTCAAACGTCGTCTGGAGGAGCTGGCGGAAGAGGCGCGTCGTAAGGCGGAAAAACTGGGTGACGAATTTCGTCTGAAGCTGGAATACGGT'

In [2]:
#Paths to your data! My data is split across 2 separate MiSeq runs, since I didn't get enough counts in the first run. 
read1_1 = '/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_NaiveLib_Characterization/XCDP07_1_R1.fastq.gz'
read2_1 = '/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_NaiveLib_Characterization/XCDP07_1_R2.fastq.gz'
read1_2 = '/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_NaiveLib_Characterization/XCDP07_2_R1.fastq.gz'
read2_2 = '/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_NaiveLib_Characterization/XCDP07_2_R2.fastq.gz'

#define the template for this library
TEMPLATE = XCDP07

#give a name for the output file!
output_name = 'XCDP07' + '_BarcodeLibrary'

In [3]:
#Define reverse complement function
alt_map = {'ins':'0'}
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
def reverse_complement(seq):    
    for k,v in alt_map.items():
        seq = seq.replace(k,v)
    bases = list(seq) 
    bases = reversed([complement.get(base,base) for base in bases])
    bases = ''.join(bases)
    for k,v in alt_map.items():
        bases = bases.replace(v,k)
    return bases
#Defines function that just reverses, no complement.
nocomplement = {'zzz': 'zzz'} 
def reverse(qual):    
    for k,v in alt_map.items():
        qual = qual.replace(k,v)
    scores = list(qual) 
    scores = reversed([nocomplement.get(score,score) for score in scores])
    scores = ''.join(scores)
    for k,v in alt_map.items():
        scores = scores.replace(v,k)
    return scores 

#Define function for finding the frequency of each base in a group and finding the consensus sequence
def find_consensus_plus_aa(group):
    
    #Set up info about template
    TEMPLATE_dna = Seq(TEMPLATE)
    TEMPLATE_aa = str(TEMPLATE_dna.translate())    
    dna_length = len(TEMPLATE)
    
    list_of_DNA_sequences = group
    #First construct a frequency matrix for each base and N
    n = max([len(dna) for dna in list_of_DNA_sequences])
    frequency_matrix = {base: np.zeros(n, dtype=np.int)
                        for base in 'ACGTN'}
    for dna in list_of_DNA_sequences:
        for index, base in enumerate(dna):
            frequency_matrix[base][index] += 1
    #Next, choose consensus based on base with highest frequency at each location
    consensus = ''
    for i in range(dna_length):  # loop over positions in string
        max_freq = -1            # holds the max freq. for this i
        max_freq_base = None     # holds the corresponding base
        for base in 'ACGT':
            if frequency_matrix[base][i] > max_freq:
                max_freq = frequency_matrix[base][i]
                max_freq_base = base
            elif frequency_matrix[base][i] == max_freq:
                max_freq_base = 'N' # more than one base as max
        consensus += max_freq_base  # add new base with max freq 
    
    consensus_dna = Seq(consensus)
    consensus_aa = str(consensus_dna.translate())         
        
    #Find the size of this group    
    group_size = size(group) 
    #find the number of unknowns and mismatches
    unknowns = consensus_aa.count('X')        
    mismatches = sum (consensus_aa[i] != TEMPLATE_aa[i] for i in range(len(TEMPLATE_aa))) - unknowns

    #Get the summary info for single point mutants (position and mutation)
    mutation = np.nan
    position = np.nan

    if mismatches == 1:
        for i in range(len(TEMPLATE_aa)):
            if consensus_aa[i] != TEMPLATE_aa[i]:
                position = i+1
                mutation = consensus_aa[i]
    if mismatches == 0:
        position = 0
        mutation = '-'
    
    #return important variables
    return [consensus_aa, group_size, mismatches, unknowns, mutation, position]

In [4]:
#Grab all of the sequencing data for read1 and read2
seqsF = []
seqsR = []
qualsF = []
qualsR = []
countF = 0
countR = 0
with gzip.open(read1_1) as f2:
    while True:
        headerF =  f2.readline()[:-1]
        seqF =  f2.readline()[:-1]
        strandF =  f2.readline()[:-1]
        qualF =  f2.readline()[:-1]
        countF = countF + 1
        if len(headerF)==0: 
            break
        seqsF.append(seqF)
        qualsF.append(qualF)
with gzip.open(read2_1) as f2:
    while True:
        headerR =  f2.readline()[:-1]
        seqR =  f2.readline()[:-1]
        strandR =  f2.readline()[:-1]
        qualR =  f2.readline()[:-1]
        countR = countR + 1
        if len(headerR)==0: 
            break
        seqsR.append(seqR)
        qualsR.append(qualR)  
seqs2F = []
seqs2R = []
quals2F = []
quals2R = []
count2F = 0
count2R = 0        
with gzip.open(read1_2) as f2:
    while True:
        headerF =  f2.readline()[:-1]
        seqF =  f2.readline()[:-1]
        strandF =  f2.readline()[:-1]
        qualF =  f2.readline()[:-1]
        count2F = count2F + 1
        if len(headerF)==0: 
            break
        seqs2F.append(seqF)
        quals2F.append(qualF)        
with gzip.open(read2_2) as f2:
    while True:
        headerR =  f2.readline()[:-1]
        seqR =  f2.readline()[:-1]
        strandR =  f2.readline()[:-1]
        qualR =  f2.readline()[:-1]
        count2R = count2R + 1
        if len(headerR)==0: 
            break
        seqs2R.append(seqR)
        quals2R.append(qualR)
        
#Combine data from both sequencing runs:
seqsF = seqsF + seqs2F
seqsR = seqsR + seqs2R
qualsF = qualsF + quals2F
qualsR = qualsR + quals2R

In [5]:
#Get dataframes in order and only keep the sequences that have correct constant regions
#convert to dataframes and reverse complement read2
read1=pd.DataFrame(data=seqsF,columns=['read1'])
read2=pd.DataFrame(data=seqsR,columns=['read2']).applymap(reverse_complement)
#Join into a single dataframe
df=read1.join(read2)
#only keep the sequences with the correct constant regions
df=df[df.read1.str.slice(19,34) == 'TCGGCTAGCCATATG']
df=df[df.read2.str.slice(185,200) == 'GGGGCGGATCCGAAC']

1083834


In [6]:
#Calculate adjustment for total length for stitching parts together
adjust = len(TEMPLATE)-76

#Separate sequences: SSM ends, overlaps, and barcodes. For each, reset index.
SSMseqPart1=pd.DataFrame(data=df.read1.str.slice(34,adjust),columns=['read1'])
df_part1=pd.DataFrame(data=SSMseqPart1.rename(columns={"read1": "part1"}),columns=['part1'])
df_part1=df_part1.reset_index()
del df_part1['index']

SSMseqPart3=pd.DataFrame(data=df.read2.str.slice(68,178),columns=['read2'])
df_part3=pd.DataFrame(data=SSMseqPart3.rename(columns={"read2": "part3"}),columns=['part3'])
df_part3=df_part3.reset_index()
del df_part3['index']

BARCODE=pd.DataFrame(data=df.read2.str.slice(257,267),columns=['read2'])
BARCODE=BARCODE.reset_index()
del BARCODE['index']

#make a copy of barcodes for easier processing after grouping
barcode2=BARCODE.copy()
barcode2.columns = ['barcodes']

#put everything into a nice new dataframe with fully assembled sequences and a defined barcode
df_all = pd.DataFrame()
df_all=df_part1.join(df_part3)
df_final = pd.DataFrame()
df_final['sequence']=df_all[['part1','part3']].apply(lambda x:''.join(x), axis=1)
df_final=df_final.join(BARCODE)
df_final=df_final.join(barcode2)

In [7]:
#Add a dummy template sequence for every possible barcode to the dataframe
#Basically, this acts as a tie breaker for cases where there are only 2 in a group
bases=['A','T','C','G']
all_possible_barcodes=[''.join(i) for i in itertools.product(bases, repeat=10)]
all_possible_barcodes2 = pd.DataFrame(data=all_possible_barcodes,columns=['read2'])
all_possible_barcodes3 = pd.DataFrame(data=all_possible_barcodes,columns=['barcodes'])

TEMPLATE_obj = [TEMPLATE]
template_list = TEMPLATE_obj * len(all_possible_barcodes2)
template_list = pd.DataFrame(data=template_list,columns=['sequence'])
no_mutations=template_list.join(all_possible_barcodes2)
no_mutations=no_mutations.join(all_possible_barcodes3)

result=pd.concat([df_final,no_mutations])
result=result.reset_index()
del result['index']

df_final = result

In [8]:
#Find the consensus sequence for each barcode. Start by only looking at sequences with 3+ barcodes
#filter out any sequences that have only 1 or 2 barcodes
filtered = df_final[df_final.groupby('read2').read2.transform(len) > 2]
#group sequences by their barcode
grouped = filtered.groupby('read2')
#find consensus AA sequence for each group
consensus_AA = grouped.sequence.apply(find_consensus_plus_aa)

In [9]:
#Assemble a dataframe from the grouped data:
#Build the summary dataframe
summary_AA = pd.DataFrame(consensus_AA.tolist(),columns=['sequence','group_size','mismatches','unknowns','mutation','position'])

#Add the barcodes to the dataframe
group_barcode = grouped.barcodes.last()
group_barcode = group_barcode.to_frame()
group_barcode = group_barcode.reset_index()
del group_barcode['read2']
summary_AA['barcode']=group_barcode

#only keep "clean" barcodes (0 or 1 mismatch, no unknowns)
summary_AA = summary_AA[summary_AA.mismatches < 2]
summary_AA = summary_AA[summary_AA.unknowns == 0]

#Fix indexing of the summary dataframe
summary_AA=summary_AA.reset_index()
#get rid of unneccessary columns
del summary_AA['index']
del summary_AA['mismatches']
del summary_AA['unknowns']
del summary_AA['sequence']

summary_AA = summary_AA.sort_values(['position','mutation','group_size'],ascending=[True,True,True])
summary_AA=summary_AA.reset_index()
del summary_AA['index']

summary_AA['pos'] = summary_AA['position']
summary_AA['pos']=summary_AA['pos'].astype(int).astype(str)

summary_AA['combined'] = summary_AA['mutation']+summary_AA['pos']

#print summary_AA
#Output file for reading into analysis script!
summary_AA.to_csv(output_name+'.csv')

In [10]:
#Other information that may be useful: I was only looking at a partial SSM library, but I still wanted to know
#what percent of the total SSM space I was covering with this library

#How many unique mutations are present in the library?
print "Total possible variants in"
print len(TEMPLATE)*20/3
print "unique mutatons"
print len(set(summary_AA['combined']))
print "percent of library coverage"
print 100*len(set(summary_AA['combined']))/(len(TEMPLATE)*20/3)

unique mutatons
1417
percent of library coverage
60
2340


In [11]:
#Want to make a histogram of group sizes?
#summary_AA
#summary_AA['group_size'].hist(bins=100)